In [4]:
%pip install ccxt

   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/5.7 MB 11.2 MB/s eta 0:00:01
   ---------------- ----------------------- 2.4/5.7 MB 6.7 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.7 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import ccxt
import math

api_key = '73661359c00e90892f167f4cbf2ae3be230e6c480b7ecbdc841bd4c7622f19df' #env에 넣을 예정
secret_key = 'a3dc3a3c04e0bf1816e0ea3b3f80d0714f4b82db94ea64a57ed2db38040e827f'

binance = ccxt.binance({
    'apiKey': api_key,
    'secret': secret_key,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',
    },
})

# 테스트넷 사용 설정
binance.set_sandbox_mode(True)

symbol = 'BTC/USDT'

# ✅ BTC/USDT 마켓 정보 가져오기
binance.load_markets()
market = binance.market(symbol)
min_qty = market['limits']['amount']['min']  # 최소 주문 수량
step_size = market['precision']['amount']   # 주문 수량 소수점 자리수
min_notional = 100  # 최소 주문 금액 (USDT)

# ✅ 현재 BTC 가격 가져오기
ticker = binance.fetch_ticker(symbol)
btc_price = ticker['last']

# 계좌 잔액 확인
balance = binance.fetch_balance()
acc_balance = balance['info']['totalWalletBalance']

# ✅ 최소 주문 수량 계산 (최소 주문 금액 100 USDT 충족)
order_min = max(min_notional / btc_price, min_qty)
order_min = round(order_min, int(abs(math.log10(step_size))))

#X = AI로 계산한 매수 시 포지션 비율. 추후 개발에 따라 매수량 임의 조정 가능
order = float(acc_balance) * 0.1
order_size = round(order/btc_price, int(abs(math.log10(step_size))))

if order_size<order_min:
    order_size += step_size
    order_size = round(order_size, int(abs(math.log10(step_size))))

while order_size * btc_price < min_notional:
    order_size += step_size
    order_size = round(order_size, int(abs(math.log10(step_size))))

print(f"BTC 현재 가격: {btc_price} USDT")
print(f"최소 주문 수량: {order_size} BTC (최소 주문 금액 {min_notional} USDT)")

# ✅ 시장가 매수 주문 실행
order = binance.create_market_buy_order(symbol, order_size)
print(order)

BTC 현재 가격: 83094.7 USDT
최소 주문 수량: 0.018 BTC (최소 주문 금액 100 USDT)
{'info': {'orderId': '4125350590', 'symbol': 'BTCUSDT', 'status': 'FILLED', 'clientOrderId': 'x-xcKtGhcue1980a6b2559abf3f0535', 'price': '0.00', 'avgPrice': '83101.20000', 'origQty': '0.018', 'executedQty': '0.018', 'cumQty': '0.018', 'cumQuote': '1495.82160', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': '0', 'updateTime': '1742362814056'}, 'id': '4125350590', 'clientOrderId': 'x-xcKtGhcue1980a6b2559abf3f0535', 'timestamp': 1742362814056, 'datetime': '2025-03-19T05:40:14.056Z', 'lastTradeTimestamp': 1742362814056, 'lastUpdateTimestamp': 1742362814056, 'symbol': 'BTC/USDT:USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': False, 'side': 'buy

In [ ]:
#매도 주문

balance = binance.fetch_balance()
positions = balance['info']['positions']
for position in positions:
    if float(position['positionAmt']) != 0:
        entire = float(position['positionAmt'])

#매도 포지션 양 결정
sell_size = entire

order = binance.create_market_sell_order(symbol, sell_size)
print(order)

{'info': {'orderId': '4125352493', 'symbol': 'BTCUSDT', 'status': 'FILLED', 'clientOrderId': 'x-xcKtGhcua05551dd2c90c847405540', 'price': '0.00', 'avgPrice': '83131.00000', 'origQty': '0.022', 'executedQty': '0.022', 'cumQty': '0.022', 'cumQuote': '1828.88200', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': '0', 'updateTime': '1742363031263'}, 'id': '4125352493', 'clientOrderId': 'x-xcKtGhcua05551dd2c90c847405540', 'timestamp': 1742363031263, 'datetime': '2025-03-19T05:43:51.263Z', 'lastTradeTimestamp': 1742363031263, 'lastUpdateTimestamp': 1742363031263, 'symbol': 'BTC/USDT:USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': False, 'side': 'sell', 'price': 83131.0, 'triggerPrice': None, 'amount': 0.022, 

In [ ]:
#현재 가격 조회
ticker = binance.fetch_ticker(symbol)
print(f"현재 가격: {ticker['last']} USDT")

#주문 내역 조회
orders = binance.fetch_orders(symbol)
for order in orders:
    print(order)